# Análise dos microdados ENADE (parte 1)
## Criando o dataset principal

In [1]:
import pandas as pd
import numpy as np

## Carrengando os datasets disponíveis no INEP (2016 a 2019)
- https://www.gov.br/inep/pt-br/acesso-a-informacao/dados-abertos/microdados

In [2]:
path='C:/BI-GOV/enade/csv/'

file_enade19 = path+'microdados_enade_2019.txt'
df_19 = pd.read_csv(file_enade19, sep=';', encoding='latin1')
file_enade18 = path+'microdados_enade_2018.txt'
df_18 = pd.read_csv(file_enade18, sep=';', encoding='latin1')
file_enade17 = path+'microdados_enade_2017.txt'
df_17 = pd.read_csv(file_enade17, sep=';', encoding='latin1')
file_enade16 = path+'microdados_enade_2016.txt'
df_16 = pd.read_csv(file_enade16, sep=';', encoding='latin1')

In [3]:
# criando a coluna de ano
df_19['ENADE_ANO'] = 2019
df_18['ENADE_ANO'] = 2018
df_17['ENADE_ANO'] = 2017
df_16['ENADE_ANO'] = 2016

In [4]:
#Verificando a quantidade de linhas e colunas nos datasets
print(df_19.shape)
print(df_18.shape)
print(df_17.shape)
print(df_16.shape)

(433930, 138)
(548127, 138)
(537436, 151)
(216044, 142)


## Resolvendo pequenas diferenças entre arquivos

In [5]:
print("16-17: ", df_16.columns.difference(df_17.columns))
print("17-16: ", df_17.columns.difference(df_16.columns))
print("16-18: ", df_16.columns.difference(df_18.columns))
print("18-16: ", df_18.columns.difference(df_16.columns))
print("16-19: ", df_16.columns.difference(df_19.columns))
print("19-16: ", df_19.columns.difference(df_16.columns))
print("17-18: ", df_17.columns.difference(df_18.columns))
print("17-19: ", df_17.columns.difference(df_19.columns))
print("18-19: ", df_18.columns.difference(df_19.columns))


16-17:  Index(['AMOSTRA', 'ANO_FIM_2G', 'ID_STATUS', 'IN_GRAD', 'IN_MATUT',
       'IN_NOTURNO', 'IN_VESPER', 'TP_SEMESTRE'],
      dtype='object')
17-16:  Index(['ANO_FIM_EM', 'CO_TURNO_GRADUACAO', 'QE_I69', 'QE_I70', 'QE_I71',
       'QE_I72', 'QE_I73', 'QE_I74', 'QE_I75', 'QE_I76', 'QE_I77', 'QE_I78',
       'QE_I79', 'QE_I80', 'QE_I81', 'TP_INSCRICAO', 'TP_INSCRICAO_ADM'],
      dtype='object')
16-18:  Index(['AMOSTRA', 'ANO_FIM_2G', 'ID_STATUS', 'IN_GRAD', 'IN_MATUT',
       'IN_NOTURNO', 'IN_VESPER', 'TP_SEMESTRE'],
      dtype='object')
18-16:  Index(['ANO_FIM_EM', 'CO_TURNO_GRADUACAO', 'TP_INSCRICAO', 'TP_INSCRICAO_ADM'], dtype='object')
16-19:  Index(['AMOSTRA', 'ANO_FIM_2G', 'ID_STATUS', 'IN_GRAD', 'IN_MATUT',
       'IN_NOTURNO', 'IN_VESPER', 'TP_SEMESTRE'],
      dtype='object')
19-16:  Index(['ANO_FIM_EM', 'CO_TURNO_GRADUACAO', 'TP_INSCRICAO', 'TP_INSCRICAO_ADM'], dtype='object')
17-18:  Index(['QE_I69', 'QE_I70', 'QE_I71', 'QE_I72', 'QE_I73', 'QE_I74', 'QE_I75',
       'Q

- As questões QE_I69 a QE_I81 são itens exclusivos para os Estudantes das Licenciaturas e só aparecem em 2017, serão removidos

- Essas colunas que só estão em 2016 serão removidas: 'AMOSTRA', 'ID_STATUS', 'TP_SEMESTRE', 'IN_GRAD'

- Renomeando ANO_FIM_2G para ANO_EM

- Alterar 'IN_MATUT','IN_NOTURNO', 'IN_VESPER' para CO_TURNO_GRADUACAO

- Colunas 'TP_INSCRICAO_ADM' e 'TP_INSCRICAO' não estão em 2016


In [6]:
#As questões QE_I69 a QE_I81 são itens exclusivos para os Estudantes das Licenciaturas e só aparecem em 2017, serão removidos
df_17.drop(columns=['QE_I69', 'QE_I70', 'QE_I71', 'QE_I72', 'QE_I73', 'QE_I74', 'QE_I75',
       'QE_I76', 'QE_I77', 'QE_I78', 'QE_I79', 'QE_I80', 'QE_I81'], inplace=True, errors='ignore')

In [7]:
# removendo as colunas a mais do dataset de 2016
df_16.drop(columns=['AMOSTRA', 'ID_STATUS', 'TP_SEMESTRE','IN_GRAD'], inplace=True, errors='ignore')

In [8]:
# Renomeando ANO_FIM_2G para ANO_FIM_EM
# Renomeando IN_GRAD para ANO_IN_GRAD
#df_16.rename(columns={'ANO_FIM_2G':'ANO_FIM_EM', 'IN_GRAD':'ANO_IN_GRAD'}, inplace=True)
df_16.rename(columns={'ANO_FIM_2G':'ANO_FIM_EM'}, inplace=True)

In [9]:
# Alterar 'IN_MATUT','IN_NOTURNO', 'IN_VESPER' para CO_TURNO_GRADUACAO e removendo essas colunas
def convert(ma,ve,no):
    if (ma+ve+no)>1:
        return 3
    if ma==1:
        return 1
    if ve==1:
        return 2
    if no==1:
        return 4
    return null

df_16['CO_TURNO_GRADUACAO'] = df_16[['IN_MATUT', 'IN_VESPER', 'IN_NOTURNO']].apply(lambda x:convert(x['IN_MATUT'], x['IN_VESPER'], x['IN_NOTURNO']), axis=1)

df_16.drop(columns=['IN_MATUT', 'IN_NOTURNO', 'IN_VESPER'], inplace=True, errors='ignore')


In [10]:
# inserindo colunas faltantes no dataset de 2016

df_16.insert(df_19.columns.get_loc('TP_INSCRICAO'), 'TP_INSCRICAO', np.nan,  allow_duplicates=False)

df_16.insert(df_19.columns.get_loc('TP_INSCRICAO_ADM'), 'TP_INSCRICAO_ADM', np.nan,  allow_duplicates=False)


In [11]:
# Verificando se removeu todas as diferenças
print("16-17: ", df_16.columns.difference(df_17.columns))
print("17-16: ", df_17.columns.difference(df_16.columns))
print("16-18: ", df_16.columns.difference(df_18.columns))
print("18-16: ", df_18.columns.difference(df_16.columns))
print("16-19: ", df_16.columns.difference(df_19.columns))
print("19-16: ", df_19.columns.difference(df_16.columns))
print("17-18: ", df_17.columns.difference(df_18.columns))
print("17-19: ", df_17.columns.difference(df_19.columns))
print("18-19: ", df_18.columns.difference(df_19.columns))

16-17:  Index([], dtype='object')
17-16:  Index([], dtype='object')
16-18:  Index([], dtype='object')
18-16:  Index([], dtype='object')
16-19:  Index([], dtype='object')
19-16:  Index([], dtype='object')
17-18:  Index([], dtype='object')
17-19:  Index([], dtype='object')
18-19:  Index([], dtype='object')


In [12]:
# Todos os dataframes tem a mesma quantidade de colunas
print("16 ", df_16.shape)
print("17 ", df_17.shape)
print("18 ", df_18.shape)
print("19 ", df_19.shape)

16  (216044, 138)
17  (537436, 138)
18  (548127, 138)
19  (433930, 138)


## Gerando o arquivo csv com todos os datasets (2016 a 2019)

In [13]:
df = pd.concat([df_16, df_17, df_18, df_19]).reset_index(drop=True)

In [14]:
# removendo linhas que possuem muitos valores null
df.dropna(how='any', thresh=20,inplace=True)

In [15]:
# Criando o arquivo csv final
df.to_csv(path+'enade16_19.csv', index=False)